# layers

This module contains many layer classes that we might be interested in using in our models. These layers complement the default [Pytorch layers](https://pytorch.org/docs/stable/nn.html) which we can also use as predefined layers.

In [ ]:
from fastai import *
from fastai.docs import *
from fastai.gen_doc.nbdoc import *

In [ ]:
show_doc(AdaptiveConcatPool2d, doc_string=False)

## <a id=AdaptiveConcatPool2d></a>`class` `AdaptiveConcatPool2d`
> `AdaptiveConcatPool2d`(`sz`:`Optional`\[`int`\]=`None`) :: [`Module`](https://pytorch.org/docs/stable/nn.html#torch.nn.Module)
<a href="https://github.com/fastai/fastai/blob/master/fastai/layers.py#L61">[source]</a>

In [ ]:
from fastai.gen_doc.nbdoc import *
from fastai.layers import * 

Layer that concats `AdaptiveAvgPool2d` and `AdaptiveMaxPool2d`. Output will be `2*sz` or 2 if `sz` is None.

The [`AdaptiveConcatPool2d`](/layers.html#AdaptiveConcatPool2d) object uses adaptive average pooling and adaptive max pooling and concatenates them both. We use this because it provides the model with the information of both methods and improves performance. This technique is called `adaptive` because it allows us to decide on what output dimensions we want, instead of choosing the input's dimensions to fit a desired output size.

Let's try training with Adaptive Average Pooling first, then with Adaptive Max Pooling and finally with the concatenation of them both to see how they fare in performance.

We will first define a [`simple_cnn`](/layers.html#simple_cnn) using [Adapative Max Pooling](https://pytorch.org/docs/stable/nn.html#torch.nn.AdaptiveMaxPool2d) by changing the source code a bit.

In [ ]:
data = get_mnist()

In [ ]:
def simple_cnn_max(actns:Collection[int], kernel_szs:Collection[int]=None,
               strides:Collection[int]=None) -> nn.Sequential:
    "CNN with `conv2d_relu` layers defined by `actns`, `kernel_szs` and `strides`"
    nl = len(actns)-1
    kernel_szs = ifnone(kernel_szs, [3]*nl)
    strides    = ifnone(strides   , [2]*nl)
    layers = [conv2d_relu(actns[i], actns[i+1], kernel_szs[i], stride=strides[i])
        for i in range(len(strides))]
    layers.append(nn.Sequential(nn.AdaptiveMaxPool2d(1), Flatten()))
    return nn.Sequential(*layers)

In [ ]:
model = simple_cnn_max((3,16,16,2))
learner = Learner(data, model, metrics=[accuracy])
learner.fit(5)

Total time: 00:11
epoch  train loss  valid loss  accuracy
0      0.096509    0.087127    0.971050  (00:02)
1      0.049341    0.057675    0.978410  (00:02)
2      0.039522    0.039750    0.986752  (00:02)
3      0.032399    0.037580    0.986752  (00:02)
4      0.023001    0.031079    0.986752  (00:02)



Now let's try with [Adapative Average Pooling](https://pytorch.org/docs/stable/nn.html#torch.nn.AdaptiveAvgPool2d) now.

In [ ]:
def simple_cnn_avg(actns:Collection[int], kernel_szs:Collection[int]=None,
               strides:Collection[int]=None) -> nn.Sequential:
    "CNN with `conv2d_relu` layers defined by `actns`, `kernel_szs` and `strides`"
    nl = len(actns)-1
    kernel_szs = ifnone(kernel_szs, [3]*nl)
    strides    = ifnone(strides   , [2]*nl)
    layers = [conv2d_relu(actns[i], actns[i+1], kernel_szs[i], stride=strides[i])
        for i in range(len(strides))]
    layers.append(nn.Sequential(nn.AdaptiveAvgPool2d(1), Flatten()))
    return nn.Sequential(*layers)

In [ ]:
model = simple_cnn_avg((3,16,16,2))
learner = Learner(data, model, metrics=[accuracy])
learner.fit(5)

Total time: 00:11
epoch  train loss  valid loss  accuracy
0      0.148990    0.120217    0.955348  (00:02)
1      0.095889    0.079068    0.972031  (00:02)
2      0.078218    0.057952    0.979392  (00:02)
3      0.051828    0.043309    0.984789  (00:02)
4      0.042995    0.038250    0.987242  (00:02)



Finally we will try with the concatenation of them both [`AdaptiveConcatPool2d`](/layers.html#AdaptiveConcatPool2d). We will see that, in fact, it increases our accuracy and decreases our loss considerably!

In [ ]:
def simple_cnn(actns:Collection[int], kernel_szs:Collection[int]=None,
               strides:Collection[int]=None) -> nn.Sequential:
    "CNN with `conv2d_relu` layers defined by `actns`, `kernel_szs` and `strides`"
    nl = len(actns)-1
    kernel_szs = ifnone(kernel_szs, [3]*nl)
    strides    = ifnone(strides   , [2]*nl)
    layers = [conv2d_relu(actns[i], actns[i+1], kernel_szs[i], stride=strides[i])
        for i in range(len(strides))]
    layers.append(nn.Sequential(AdaptiveConcatPool2d(1), Flatten()))
    return nn.Sequential(*layers)

In [ ]:
model = simple_cnn((3,16,16,2))
learner = Learner(data, model, metrics=[accuracy])
learner.fit(5)

Total time: 00:11
epoch  train loss  valid loss  accuracy
0      0.067155    0.053196    0.981354  (00:02)
1      0.031042    0.027170    0.991659  (00:02)
2      0.025594    0.026606    0.990677  (00:02)
3      0.017740    0.016498    0.994112  (00:02)
4      0.018267    0.018616    0.993621  (00:02)



In [ ]:
show_doc(Lambda, doc_string=False)

## <a id=Lambda></a>`class` `Lambda`
> `Lambda`(`func`:`LambdaFunc`) :: [`Module`](https://pytorch.org/docs/stable/nn.html#torch.nn.Module)
<a href="https://github.com/fastai/fastai/blob/master/fastai/layers.py#L8">[source]</a>

Lambda allows us to define functions and use them as layers in our networks inside a [Sequential](https://pytorch.org/docs/stable/nn.html#torch.nn.Sequential) object. 

So, for example, say we want to apply a [log_softmax loss](https://pytorch.org/docs/stable/nn.html#torch.nn.functional.log_softmax) and we need to change the shape of our output batches to be able to use this loss. We can add a layer that applies the necessary change in shape by calling:

`Lambda(lambda x: x.view(x.size(0),-1))`

Let's see an example of how the shape of our output can change when we add this layer.

In [ ]:
model = nn.Sequential(
    nn.Conv2d(3,  16, kernel_size=3, stride=2, padding=1), nn.ReLU(),
    nn.Conv2d(16, 16, kernel_size=3, stride=2, padding=1), nn.ReLU(),
    nn.Conv2d(16, 10, kernel_size=3, stride=2, padding=1), nn.ReLU(),
    nn.AdaptiveAvgPool2d(1),
)

model.cuda()

for xb, yb in data.train_dl:
    out = (model(*[xb]))
    print(out.size())
    break

torch.Size([64, 10, 1, 1])


In [ ]:
model = nn.Sequential(
    nn.Conv2d(3,  16, kernel_size=3, stride=2, padding=1), nn.ReLU(),
    nn.Conv2d(16, 16, kernel_size=3, stride=2, padding=1), nn.ReLU(),
    nn.Conv2d(16, 10, kernel_size=3, stride=2, padding=1), nn.ReLU(),
    nn.AdaptiveAvgPool2d(1),
    Lambda(lambda x: x.view(x.size(0),-1))
)

model.cuda()

for xb, yb in data.train_dl:
    out = (model(*[xb]))
    print(out.size())
    break

torch.Size([64, 10])


In [ ]:
show_doc(Flatten)

#### <a id=Flatten></a>`Flatten`
> `Flatten`() -> `Tensor`


Flattens `x` to a single dimension, often used at the end of a model. <a href="https://github.com/fastai/fastai/blob/master/fastai/layers.py#L21">[source]</a>

The function we build above is actually implemented in our library as [`Flatten`](/layers.html#Flatten). We can see that it returns the same size when we run it.

In [ ]:
model = nn.Sequential(
    nn.Conv2d(3,  16, kernel_size=3, stride=2, padding=1), nn.ReLU(),
    nn.Conv2d(16, 16, kernel_size=3, stride=2, padding=1), nn.ReLU(),
    nn.Conv2d(16, 10, kernel_size=3, stride=2, padding=1), nn.ReLU(),
    nn.AdaptiveAvgPool2d(1),
    Flatten(),
)

model.cuda()

for xb, yb in data.train_dl:
    out = (model(*[xb]))
    print(out.size())
    break

torch.Size([64, 10])


In [ ]:
show_doc(PoolFlatten)

#### <a id=PoolFlatten></a>`PoolFlatten`
> `PoolFlatten`() -> [`Sequential`](https://pytorch.org/docs/stable/nn.html#torch.nn.Sequential)


Apply `nn.AdaptiveAvgPool2d` to `x` and then flatten the result. <a href="https://github.com/fastai/fastai/blob/master/fastai/layers.py#L25">[source]</a>

We can combine these two final layers ([AdaptiveAvgPool2d](https://pytorch.org/docs/stable/nn.html#torch.nn.AdaptiveAvgPool2d) and [`Flatten`](/layers.html#Flatten)) by using [`Pool Flatten`](/layers.html#Pool Flatten).

In [ ]:
model = nn.Sequential(
    nn.Conv2d(3,  16, kernel_size=3, stride=2, padding=1), nn.ReLU(),
    nn.Conv2d(16, 16, kernel_size=3, stride=2, padding=1), nn.ReLU(),
    nn.Conv2d(16, 10, kernel_size=3, stride=2, padding=1), nn.ReLU(),
    PoolFlatten()
)

model.cuda()

for xb, yb in data.train_dl:
    out = (model(*[xb]))
    print(out.size())
    break

torch.Size([64, 10])


In [ ]:
show_doc(ResizeBatch)

#### <a id=ResizeBatch></a>`ResizeBatch`
> `ResizeBatch`(`size`:`int`) -> `Tensor`


Layer that resizes x to `size`, good for connecting mismatched layers. <a href="https://github.com/fastai/fastai/blob/master/fastai/layers.py#L17">[source]</a>

Another use we give to the Lambda function is to resize batches with [`ResizeBatch`](/layers.html#ResizeBatch) when we have a layer that expects a different input than what comes from the previous one. Let's see an example:

In [ ]:
a = torch.tensor([[1., -1.], [1., -1.]])
print(a)

tensor([[ 1., -1.],
        [ 1., -1.]])


In [ ]:
out = ResizeBatch(4)
print(out(a))

tensor([[ 1., -1.,  1., -1.]])


In [ ]:
show_doc(StdUpsample, doc_string=False)

## <a id=StdUpsample></a>`class` `StdUpsample`
> `StdUpsample`(`n_in`:`int`, `n_out`:`int`) :: [`Module`](https://pytorch.org/docs/stable/nn.html#torch.nn.Module)
<a href="https://github.com/fastai/fastai/blob/master/fastai/layers.py#L76">[source]</a>

Increases the dimensionality of our data from `n_in` to `n_out` by applying a transposed convolution layer to the input and with batchnorm and a RELU activation.

In [ ]:
show_doc(CrossEntropyFlat, doc_string=False)

## <a id=CrossEntropyFlat></a>`class` `CrossEntropyFlat`
> `CrossEntropyFlat`(`weight`=`None`, `size_average`=`None`, `ignore_index`=`-100`, `reduce`=`None`, `reduction`=`'elementwise_mean'`) :: [`CrossEntropyLoss`](https://pytorch.org/docs/stable/nn.html#torch.nn.CrossEntropyLoss)
<a href="https://github.com/fastai/fastai/blob/master/fastai/layers.py#L94">[source]</a>

Same as [nn.CrossEntropyLoss](https://pytorch.org/docs/stable/nn.html#torch.nn.CrossEntropyLoss), but flattens input and target. Is used to calculate cross entropy on arrays (which Pytorch will not let us do with their [nn.CrossEntropyLoss](https://pytorch.org/docs/stable/nn.html#torch.nn.CrossEntropyLoss) function). An example of a use case is image segmentation models where the output in an image (or an array of pixels).

The parameters are the same as [nn.CrossEntropyLoss](https://pytorch.org/docs/stable/nn.html#torch.nn.CrossEntropyLoss): `weight` to rescale each class, `size_average` whether we want to sum the losses across elements in a batch or we want to add them up, `ignore_index` what targets do we want to ignore, `reduce` on whether we want to return a loss per batch element and `reduction` specifies which type of reduction (if any) we want to apply to our input.

In [ ]:
show_doc(Debugger)

## <a id=Debugger></a>`class` `Debugger`
> `Debugger`() :: [`Module`](https://pytorch.org/docs/stable/nn.html#torch.nn.Module)


A module to debug inside a model. <a href="https://github.com/fastai/fastai/blob/master/fastai/layers.py#L70">[source]</a>

The debugger module allows us to peek inside a network while its training and see in detail what is going on. We can see inputs, ouputs and sizes at any point in the network.

For instance, if you run the following:

``` python
model = nn.Sequential(
    nn.Conv2d(3,  16, kernel_size=3, stride=2, padding=1), nn.ReLU(),
    Debugger(),
    nn.Conv2d(16, 16, kernel_size=3, stride=2, padding=1), nn.ReLU(),
    nn.Conv2d(16, 10, kernel_size=3, stride=2, padding=1), nn.ReLU(),
)

model.cuda()

learner = Learner(data, model, metrics=[accuracy])
learner.fit(5)
```
... you'll see something like this:

```
/home/ubuntu/fastai/fastai/layers.py(74)forward()
     72     def forward(self,x:Tensor) -> Tensor:
     73         set_trace()
---> 74         return x
     75 
     76 class StdUpsample(nn.Module):

ipdb>
```

In [ ]:
show_doc(bn_drop_lin, doc_string=False)

#### <a id=bn_drop_lin></a>`bn_drop_lin`
> `bn_drop_lin`(`n_in`:`int`, `n_out`:`int`, `bn`:`bool`=`True`, `p`:`float`=`0.0`, `actn`:`Optional`\[[`Module`](https://pytorch.org/docs/stable/nn.html#torch.nn.Module)\]=`None`)
<a href="https://github.com/fastai/fastai/blob/master/fastai/layers.py#L29">[source]</a>

The [`bn_drop_lin`](/layers.html#bn_drop_lin) function returns a sequence of [batch normalization](https://arxiv.org/abs/1502.03167), [dropout](https://www.cs.toronto.edu/~hinton/absps/JMLRdropout.pdf) and a linear layer. This custom layer is usually used at the end of a model. 

`n_in` represents the number of size of the input `n_out` the size of the output, `bn` whether we want batch norm or not, `p` is how much dropout and `actn` is an optional parameter to add an activation function at the end.

In [ ]:
show_doc(conv2d)

#### <a id=conv2d></a>`conv2d`
> `conv2d`(`ni`:`int`, `nf`:`int`, `ks`:`int`=`3`, `stride`:`int`=`1`, `padding`:`int`=`None`, `bias`=`False`) -> [`Conv2d`](https://pytorch.org/docs/stable/nn.html#torch.nn.Conv2d)


Create `nn.Conv2d` layer: `ni` inputs, `nf` outputs, `ks` kernel size. `padding` defaults to `k//2`. <a href="https://github.com/fastai/fastai/blob/master/fastai/layers.py#L37">[source]</a>

In [ ]:
show_doc(conv2d_relu, doc_string=False)

#### <a id=conv2d_relu></a>`conv2d_relu`
> `conv2d_relu`(`ni`:`int`, `nf`:`int`, `ks`:`int`=`3`, `stride`:`int`=`1`, `padding`:`int`=`None`, `bn`:`bool`=`False`) -> [`Sequential`](https://pytorch.org/docs/stable/nn.html#torch.nn.Sequential)
<a href="https://github.com/fastai/fastai/blob/master/fastai/layers.py#L49">[source]</a>

Create a [`conv2d`](/layers.html#conv2d) layer with `nn.ReLU` activation and optional(`bn`) `nn.BatchNorm2d`: `ni` input, `nf` out filters, `ks` kernel, `stride`:stride, `padding`:padding, `bn`: batch normalization.

In [ ]:
show_doc(conv2d_trans)

#### <a id=conv2d_trans></a>`conv2d_trans`
> `conv2d_trans`(`ni`:`int`, `nf`:`int`, `ks`:`int`=`2`, `stride`:`int`=`2`, `padding`:`int`=`0`) -> [`ConvTranspose2d`](https://pytorch.org/docs/stable/nn.html#torch.nn.ConvTranspose2d)


Create `nn.ConvTranspose2d` layer: `ni` inputs, `nf` outputs, `ks` kernel size, `stride`: stride. `padding` defaults to 0. <a href="https://github.com/fastai/fastai/blob/master/fastai/layers.py#L57">[source]</a>

In [ ]:
show_doc(conv_layer, doc_string=False)

#### <a id=conv_layer></a>`conv_layer`
> `conv_layer`(`ni`:`int`, `nf`:`int`, `ks`:`int`=`3`, `stride`:`int`=`1`) -> [`Sequential`](https://pytorch.org/docs/stable/nn.html#torch.nn.Sequential)
<a href="https://github.com/fastai/fastai/blob/master/fastai/layers.py#L42">[source]</a>

The [`conv_layer`](/layers.html#conv_layer) function returns a sequence of [nn.Conv2D](https://pytorch.org/docs/stable/nn.html#torch.nn.Conv2d), [BatchNorm2d](https://arxiv.org/abs/1502.03167) and a [leaky RELU](https://ai.stanford.edu/~amaas/papers/relu_hybrid_icml2013_final.pdf) activation function.

`n_in` represents the number of size of the input `n_out` the size of the output, `ks` kernel size, `stride` the stride with which we want to apply the convolutions.

In [ ]:
show_doc(get_embedding, doc_string=False)

#### <a id=get_embedding></a>`get_embedding`
> `get_embedding`(`ni`:`int`, `nf`:`int`) -> [`Module`](https://pytorch.org/docs/stable/nn.html#torch.nn.Module)
<a href="https://github.com/fastai/fastai/blob/master/fastai/layers.py#L116">[source]</a>

Create an [embedding layer](https://arxiv.org/abs/1711.09160) with input size `ni` and output size `nf`.

In [ ]:
show_doc(simple_cnn)

#### <a id=simple_cnn></a>`simple_cnn`
> `simple_cnn`(`actns`:`Collection`\[`int`\], `kernel_szs`:`Collection`\[`int`\]=`None`, `strides`:`Collection`\[`int`\]=`None`) -> [`Sequential`](https://pytorch.org/docs/stable/nn.html#torch.nn.Sequential)


CNN with [`conv2d_relu`](/layers.html#conv2d_relu) layers defined by `actns`, `kernel_szs` and `strides`. <a href="https://github.com/fastai/fastai/blob/master/fastai/layers.py#L100">[source]</a>

In [ ]:
show_doc(std_upsample_head, doc_string=False)

#### <a id=std_upsample_head></a>`std_upsample_head`
> `std_upsample_head`(`c`, `nfs`:`Collection`\[`int`\]) -> [`Module`](https://pytorch.org/docs/stable/nn.html#torch.nn.Module)
<a href="https://github.com/fastai/fastai/blob/master/fastai/layers.py#L86">[source]</a>

Create a sequence of upsample layers with a RELU at the beggining and a [nn.ConvTranspose2d](https://pytorch.org/docs/stable/nn.html#torch.nn.ConvTranspose2d). 

`nfs` is a list with the input and output sizes of each upsample layer and `c` is the output size of the final 2D Transpose Convolutional layer.

In [ ]:
show_doc(trunc_normal_)

#### <a id=trunc_normal_></a>`trunc_normal_`
> `trunc_normal_`(`x`:`Tensor`, `mean`:`float`=`0.0`, `std`:`float`=`1.0`) -> `Tensor`


Truncated normal initialization. <a href="https://github.com/fastai/fastai/blob/master/fastai/layers.py#L111">[source]</a>